In [ ]:
import numpy as np
import pandas as pd
import scipy.stats

pd.options.mode.chained_assignment = None
from matplotlib import pyplot as plt, rcParams
# import cv2
import seaborn as sns

sns.set(style="white", context="paper")
from cycler import cycler
import os, sys
import glob
from datetime import datetime, timedelta
from itertools import combinations
import base64
from PIL import Image
from io import BytesIO as _BytesIO
import requests
import json
import pickle
from datetime import datetime
from IPython.display import display, Markdown, Latex
from sklearn.metrics import *
import collections
from copy import deepcopy
pd.options.display.max_columns = None
def printm(s): return display(Markdown(s))


## Configs for data fetch and plotting

In [23]:
overall_cache_dir = "../../cache/sep12_inc/"
# Result summary prefix
result_summary_prefix = 'results_summary'

#timestamped predictions prefix
ts_prediction_file_prefix = 'timestamped_predictions'

# Ground Truth
gt_dir = f'../../GT_marking/labelstudio_gt/'


In [24]:
## config for various kinds of graphs

gconfigs = {
    'barplot': {'color': 'blue', 'linestyle': '-.', 'marker': '.', 'alpha': 0.5}
}


#Percentile calculations
def perc_75(x): return np.percentile(x, 75)


def perc_25(x): return np.percentile(x, 25)


#Set default RC parameters
notebook_default_rcparams = {
    "axes.titlesize": 32,
    "axes.labelsize": 32,
    "legend.fontsize": 32,
    "legend.title_fontsize": 32,
    "xtick.labelsize": 32,
    "ytick.labelsize": 32,
    "axes.grid": True,
    "legend.framealpha": 0.5,
    "lines.linewidth": 5,
    "legend.loc": 'upper left'

}
rcParams.update(notebook_default_rcparams)

# Standardized Labels

EPSILON = 2e-2
#plotting dir


plotting_dir = f'plots/{datetime.now().strftime("%Y%m%d")}'
if not os.path.exists(plotting_dir):
    os.makedirs(plotting_dir)
out_result_dir = f'results/{datetime.now().strftime("%Y%m%d")}'
if not os.path.exists(out_result_dir):
    os.makedirs(out_result_dir)


# Ground truth collection

In [25]:
gt_dict = {}
if True:
    df_gt = None
    gt_ids = glob.glob(f"{gt_dir}/*.json")
    list_gt_raw=[]
    for id_file in gt_ids:
        # print(f"collecting labels for id file {id_file}")
        id_gt_dict = json.load(open(id_file,'r'))
        id = id_file.split("/")[-1].split(".json")[0]
        for task_idx in range(len(id_gt_dict)):
            task_url = id_gt_dict[task_idx]['timeseriesUrl'].split("/")[-1].split(".")[0]
            label_info = id_gt_dict[task_idx]['label']
            for label_dict in label_info:
                label_start,label_end = label_dict['start'], label_dict['end']
                # print("Before conversion",task_url,label_start,label_end)
                try:
                    label_start = pd.to_datetime(label_start,format="%Y-%m-%d %H:%M:%S" )
                    label_end = pd.to_datetime(label_end,format="%Y-%m-%d %H:%M:%S" )
                    if label_dict['instant']:
                        label_end = label_end + timedelta(seconds=1)
                    if not (id[:3]=='csh'): #it's and extrasensory id
                        label_start = label_start.tz_localize('America/Los_Angeles')
                        label_end = label_end.tz_localize('America/Los_Angeles')
                    label_start_timestamp = label_start.timestamp()
                    label_end_timestamp = label_end.timestamp()
                    label_value = ",".join(label_dict['timeserieslabels'])
                    list_gt_raw.append([id,task_url,label_start_timestamp,label_end_timestamp,label_value])
                except:
                    # print(label_dict)
                    ...
    df_gt = pd.DataFrame(np.array(list_gt_raw),columns=['id','task_url','start_time','end_time','context'])
    df_gt['start_time'] = df_gt['start_time'].astype(float)
    df_gt['end_time'] = df_gt['end_time'].astype(float)
    df_gt['context'] = df_gt['context'].apply(lambda x: 'Amusement' if (x=='1') else x)
    df_gt['context'] = df_gt['context'].apply(lambda x: 'Commuting' if (x=='2') else x)
    df_gt['context'] = df_gt['context'].apply(lambda x: 'Exercising' if (x=='3') else x)
    df_gt['context'] = df_gt['context'].apply(lambda x: 'GoingOut' if (x=='4') else x)
    df_gt['context'] = df_gt['context'].apply(lambda x: 'HavingMeal' if (x=='a') else x)
    gt_dict['instance_counts'] = df_gt['context'].value_counts()
    df_gt.id.drop_duplicates().values.tolist()
    df_gt['total_time'] = df_gt['end_time'].astype(float) - df_gt['start_time'].astype(float)
    gt_dict['total_times'] = df_gt.groupby('context')['total_time'].sum() / 3600.
    # df_gt[df_gt.id.isin(['csh101','csh102'])].groupby('context')['total_time'].sum().sort_index() / 3600.
# df_gt.info()
df_gt.head()

,id,task_url,start_time,end_time,context,total_time
0,00EABED2-271D-49D8-B599-1D4A09240601,a21f9208-2015-10-07,1.444252e+09,1.444252e+09,Commuting,1.0
1,00EABED2-271D-49D8-B599-1D4A09240601,a21f9208-2015-10-07,1.444233e+09,1.444234e+09,Commuting,480.0
2,00EABED2-271D-49D8-B599-1D4A09240601,a21f9208-2015-10-07,1.444249e+09,1.444249e+09,InAMeeting,1.0
3,00EABED2-271D-49D8-B599-1D4A09240601,07ac640a-2015-10-08,1.444348e+09,1.444374e+09,Sleeping,25651.0
4,00EABED2-271D-49D8-B599-1D4A09240601,07ac640a-2015-10-08,1.444341e+09,1.444341e+09,Sleeping,1.0


# Ontology, Ontological predictions and Helper functions

In [26]:
 # Onotlogy Graphs and Labeling contexts with ontology
if True:

    df_onto = pd.read_csv('../../ontological_models/ontology_labels_aug11.csv', names=['activities', 'contexts'])
    df_onto['contexts'] = df_onto['contexts'].apply(lambda x: x.split(";"))
    onto_dict = df_onto.set_index('activities').to_dict()['contexts']

    df_onto_pred = pd.read_csv('../../ontological_models/ontology_predictions_v2.csv')
    df_onto_pred['tao_prediction'] = df_onto_pred['tao_prediction'].apply(lambda x: x.split(";") if not (str(x)=='nan') else ['Unknown'])
    # df_onto_pred = pd.read_csv('../../cache/df_gt_with_activities_aug12.csv')
    # df_onto_pred['tao_prediction'] = df_onto_pred['gt_context'].apply(lambda x: x.split(",") if not (str(x)=='nan') else ['Unknown'])


    activity_rename_mapping = {
        'extrasensory': {
        'lying': 'LyingDown',
        'sitting': 'Sitting',
        'walking': 'Walking',
        'running': 'Running',
        'cycling': 'Cycling',
        'sleeping': 'Sleeping',
        'meeting': 'Meeting',
        'driving': 'Driving',
        'exercising': 'Hiking',
        'cooking': 'Cooking',
        'shopping': 'Shopping',
        'drinking': 'Drinking',
        'shower': 'Shower',
        'cleaning': 'VacuumHome',
        'laundry': 'VacuumHome',
        'clean_dishes': 'VacuumHome',
        'watching_tv': 'WatchingTv',
        'surfing_internet': 'ReadingOffice',
        'singing': 'Dancing',
        'talking': 'Talking',
        'office_work': 'TypingOffice',
        'eating': 'Eating',
        'toilet': 'Toilet',
        'grooming': 'Grooming',
        'dressing_up': 'Grooming',
        'stairs': 'ClimbingStairs',
        'standing': 'Standing',
        'meeting_coworkers': 'Meeting',
        'meeting_friends': 'Dancing',

    },
        'casas': {
            'step_out': 'StepOut',
            'none': 'None',
            'toilet': 'Toilet',
            'onphone': 'OnPhone',
            'grooming': 'Grooming',
            'step_in': 'StepIn',
            'lying': 'LyingDown',
            'drinking': 'Drinking',
            'watching_tv': 'WatchingTv',
            'dressing_up': 'Grooming',
            'taking_meds': 'Eating',
            'wakingup': 'Sleeping',
            'reading': 'TypingOffice',
            'cooking': 'Cooking',
            'eating': 'Eating',
            'shower': 'Shower',
            'sleeping': 'Sleeping',
            'office_work': 'SittingOffice',
            'dishes_home': 'VacuumHome',
            'meeting_friends': 'Dancing',
            'exercising': 'Running',
            'laundry_home': 'VacuumHome'
        },
        'tsu': {
            "boil_water": "Cooking",
            "clean_with_water": "VacuumHome",
            "cut_cook": "Cooking",
            "cut_bread": "Cooking",
            "drink_cold": "Drinking",
            "drink_hot": "Drinking",
            "dry_up": "VacuumHome",
            "dump_in_trash": "VacuumHome",
            "eat_food": "Eating",
            "eat_snack": "Eating",
            "enter": "StepIn",
            "get_up": "Sleeping",
            "get_water": "Eating",
            "insert_tea_bag": "Drinking",
            "lay_down": "LyingDown",
            "leave": "StepOut",
            "pour_grains": "Drinking",
            "pour_water": "Drinking",
            "pour_cold": "Drinking",
            "pour_hot": "Drinking",
            "put_in_sink": "VacuumHome",
            "put_on_table": "VacuumHome",
            "read": "ReadingOffice",
            "sit_down": "Sitting",
            "spread_jam_or_butter": "Eating",
            "stir_cook": "Cooking",
            "stir_drink": "Drinking",
            "take_ham": "Cooking",
            "take_meds": "Eating",
            "take_off_table": "Eating",
            "use_furniture": "VacuumHome",
            "use_glasses": "Eating",
            "use_pc": "TypingOffice",
            "use_kitchen_utility": "VacuumHome",
            "use_telephone": "onPhone",
            "walk": "Walking",
            "watch_tv": "WatchingTv",
            "clean_table": "VacuumHome",
            "write": "ReadingOffice"
        }
    }

    tsu_context_mapping = {
        "ComingIn": ["tsuBreakfast"],
        "Commuting": ["tsuBreakfast", "tsuCook"],
        "GoingOut": ["tsuBreakfast"],
        "HavingMeal": ["tsuMakecoffee", "tsuMaketea"],
        "HouseWork": ["tsuCleandishes"],
        "PreparingMeal": ["tsuCook"],
        "Relaxing": ["tsuBreakfast"],
    }

def label_context_v1(cluster_representation, activity_renaming,conf_activities=['sitting','standing','talking']):
    # print(cluster_representation)
    act_train = cluster_representation.split(">")
    act_train = [xr.split("+") for xr in act_train]

    unique_activities = set()
    for act_set in act_train:
        for activity in act_set:
            unique_activities.add(activity)
    for conf_act in conf_activities:
        try:
            unique_activities.remove(conf_act)
        except:
            ...


    if len(unique_activities)>0:
        # print("More than conf activities in the set, removing conf activities for precise context labeling")
        act_train = [([activity for activity in act_set if (activity not in conf_activities)]) for act_set in act_train]
        # print(act_train)


    for i in range(len(act_train)):
        for j in range(len(act_train[i])):
            # print(act_train,act_train[i], act_train[i][j])
            if not act_train[i][j]=='unknown':
                act_train[i][j] = activity_renaming[act_train[i][j]].lower()
            else:
                act_train[i][j] = 'none'
        act_train[i] = sorted(np.unique(act_train[i]).tolist())

    #for sequential contexts
    # try:
    seq_contexts = []
    for i in range(1, len(act_train)):
        set1, set2 = act_train[i - 1], act_train[i]
        for first_act in set1:
            for sec_act in set2:
                # print(f"seq: {first_act},{sec_act}")
                if not first_act == sec_act:
                    try:
                        seq_ctx = onto_dict[
                            f'{first_act}+{sec_act}']
                        if not (seq_ctx[0] == 'Unknown'):
                            seq_contexts += seq_ctx
                    except:
                        ...

    # for parallel contexts
    single_act_contexts = []
    par_contexts = []
    for act_set in act_train:
        if len(act_set) == 1:
            single_ctx = onto_dict[f"{act_set[0]}"]
            if not (single_ctx[0] == 'Unknown'):
                single_act_contexts += single_ctx
        else:
            for act1, act2 in combinations(act_set, 2):
                if not act1 == act2:
                    par_ctx = ['Unknown']
                    try:
                        par_ctx = onto_dict[f"{act1}_{act2}"]
                    except:
                        par_ctx = onto_dict[f"{act2}_{act1}"]
                    if not (par_ctx[0] == 'Unknown'):
                        par_contexts += par_ctx

    final_context_set = None
    if len(seq_contexts) > 0:
        final_context_set = np.unique(seq_contexts).tolist()
    elif len(par_contexts) > 0:
        final_context_set = np.unique(par_contexts).tolist()
    elif len(single_act_contexts) > 0:
        final_context_set = np.unique(single_act_contexts).tolist()
    else:
        all_contexts = []
        for act_set in act_train:
            for activity in act_set:
                single_ctx = onto_dict[f"{activity}"]
                if not (single_ctx[0] == 'Unknown'):
                    all_contexts += single_ctx
        final_context_set = np.unique(all_contexts).tolist()

    # tsu specific conversion
    if dataset == 'tsu':
        tsu_final_contexts = []
        for context in final_context_set:
            if context in tsu_context_mapping.keys():
                tsu_final_contexts += tsu_context_mapping[context]
        if len(tsu_final_contexts) > 0:
            return np.unique(tsu_final_contexts).tolist()
        else:
            return ['Unknown']
    else:
        return final_context_set

In [32]:
# helper functions

def get_ctx_vec(ctx_str, contexts):
    ctx_vec = np.zeros(len(contexts))
    for item_i in ctx_str.split(","):
        if not item_i in ['','Unknown']:
            ctx_vec[contexts.index(item_i)] = 1
    return ctx_vec

def get_cluster_labels(cluster_centers, dataset, labeling_func=label_context_v1):
    cluster_centers = [center.split(")__")[0].split("(")[-1] for center in cluster_centers]
    cluster_labels = [labeling_func(center, activity_rename_mapping[dataset]) for center in cluster_centers]
    return cluster_labels

def compile_ts_results_v2(ts_results, df_onto_pred, df_gt, cluster_labels):
    #compiled final timestamped results with GT
    compiled_ts_results_dict = dict()
    # compiled_instance_results_dict = dict()
    for key in ts_results.keys():
        df_pred_id = ts_results[key]
        df_pred_id['id'] = key
        df_gt_id = df_gt[df_gt.id == key]
        df_onto_pred_id = df_onto_pred[df_onto_pred.id==key]
        df_pred_id = df_pred_id[df_pred_id.end_timestamp >= df_gt_id.start_time.min()]
        df_pred_id = df_pred_id[df_pred_id.start_timestamp <= df_gt_id.end_time.max()]
        if (df_gt_id.shape[0] > 0) & (df_pred_id.shape[0] > 0) & (df_onto_pred_id.shape[0] > 0):
            # if (df_gt_id.shape[0] > 0) & (df_pred_id.shape[0] > 0):
            print(f"Starting on {key}")
            #process gt to timestamp, context format
            gt_min_ts, gt_max_ts = df_gt_id.start_time.min(), df_gt_id.end_time.max()
            df_gt_ts = pd.DataFrame(np.arange(gt_min_ts, gt_max_ts + 1), columns=['timestamp'])
            df_gt_ts['gt_context'] = ''
            df_gt_ts = df_gt_ts.set_index('timestamp')
            for row_idx, row in df_gt_id.iterrows():
                df_gt_ts.loc[row['start_time']:row['end_time'], 'gt_context'] = df_gt_ts.loc[
                                                                                row['start_time']:row['end_time'],
                                                                                'gt_context'].apply(
                    lambda x: row['context'] if (x == '') else (x + ',' + row['context']))

            #get ts based prediction
            pred_min_ts, pred_max_ts = df_pred_id.start_timestamp.min(), df_pred_id.end_timestamp.max()
            df_pred_ts = pd.DataFrame(np.arange(pred_min_ts, pred_max_ts + 1), columns=['timestamp'])
            df_pred_ts['pred_context'] = None
            df_pred_ts = df_pred_ts.set_index('timestamp')
            for row_idx, row in df_pred_id.iterrows():
                df_pred_ts.loc[row['start_timestamp']:row['end_timestamp'], 'pred_context'] = df_pred_ts.loc[
                                                                                              row['start_timestamp']:
                                                                                              row['end_timestamp'],
                                                                                              'pred_context'].apply(
                    lambda x: (x + cluster_labels[row['cluster_id']]) if x is not None else cluster_labels[
                        row['cluster_id']])


            #merge timestaped gt and predictions together for the user
            df_compiled_results_ts_id = pd.merge(df_pred_ts.reset_index(), df_gt_ts.reset_index(), on='timestamp')
            df_compiled_results_ts_id['id'] = key
            df_compiled_results_ts_id = df_compiled_results_ts_id[
                ['id', 'timestamp', 'gt_context', 'pred_context']]
            df_compiled_results_ts_id = df_compiled_results_ts_id[~df_compiled_results_ts_id.gt_context.isnull()]
            df_compiled_results_ts_id = df_compiled_results_ts_id[~df_compiled_results_ts_id.pred_context.isnull()]
            # df_compiled_results_ts_id = pd.merge(df_compiled_results_ts_id,df_onto_pred_id,on =['id','timestamp'],suffixes=('','_onto'))
            # df_compiled_results_ts_id['combined_context'] = df_compiled_results_ts_id.apply(lambda row: row['pred_context']+row['tao_prediction'],axis=1)

            #added because no results for ontology
            df_compiled_results_ts_id = pd.merge(df_compiled_results_ts_id,df_onto_pred_id,on =['id','timestamp'],suffixes=('','_onto'),how='left')
            df_compiled_results_ts_id['tao_prediction'] = df_compiled_results_ts_id['tao_prediction'].apply(lambda x: ['Unknown'] if (str(x)=='nan') else x)
            df_compiled_results_ts_id['combined_context'] = df_compiled_results_ts_id.apply(lambda row: row['pred_context'],axis=1)



            # format as a comma separated string
            df_compiled_results_ts_id['onto_context'] = df_compiled_results_ts_id['tao_prediction'].apply(
                lambda x: ','.join(sorted(np.unique(x).tolist())))
            df_compiled_results_ts_id['pred_context'] = df_compiled_results_ts_id['pred_context'].apply(
                lambda x: ','.join(sorted(np.unique(x).tolist())))
            df_compiled_results_ts_id['combined_context'] = df_compiled_results_ts_id['combined_context'].apply(
                lambda x: ','.join(sorted(np.unique(x).tolist())))

            #todo: added to deactivate ontology
            # df_compiled_results_ts_id['onto_context'] = df_compiled_results_ts_id['pred_context']
            # df_compiled_results_ts_id['combined_context'] = df_compiled_results_ts_id['pred_context']

            compiled_ts_results_dict[key] = df_compiled_results_ts_id
            print(f"Processed for id {key}")
    return compiled_ts_results_dict

def find_all_present_contexts(df_gt_ts):
    '''

    :param df_gt_ts:
    :type df_gt_ts:
    :return:
    :rtype:
    '''
    context_set = set()
    for context in df_gt_ts['gt_context'].unique():
        for item in context.split(","):
            context_set.add(item)

    for context in df_gt_ts['pred_context'].unique():
        for item in context.split(","):
            context_set.add(item)

    context_set.remove('')
    all_contexts = sorted(list(context_set))
    return all_contexts

def get_overall_metrics(gt_arr, pred_arr):

    # calculate spot by converting it into +0.2 JC
    gt_or_pred = np.sum(np.logical_or(gt_arr,pred_arr),axis=1)
    gt_eq_pred = np.sum(np.logical_and(gt_arr,pred_arr),axis=1)
    jc_samples = gt_eq_pred/gt_or_pred
    is_spot = jc_samples>0.2
    spot_acc = round(is_spot.sum()*100/is_spot.shape[0],2)
    # gt_xor_pred = np.sum(np.logical_xor(gt_arr,pred_arr),axis=1)
    # is_norm = (gt_xor_pred==0)
    # norm_acc = round(is_norm.sum()*100/is_norm.shape[0],2)



    jc_score_samples = round(jaccard_score(gt_arr, pred_arr, average='samples',zero_division=0) * 100, 2)
    f1_acc =round(f1_score(gt_arr, pred_arr, average='weighted',zero_division=0) * 100, 2)
    avg_prec = round(precision_score(gt_arr, pred_arr, average='weighted',zero_division=0) * 100, 2)
    avg_recall = round(recall_score(gt_arr, pred_arr, average='weighted',zero_division=0) * 100, 2)
    avg_f1 = round(f1_score(gt_arr, pred_arr, average='weighted',zero_division=0) * 100, 2)
    return f1_acc, avg_prec, avg_recall, jc_score_samples


# Loop over experiments to create overarching result csv

In [38]:

experiment_dirs = glob.glob(f'{overall_cache_dir}/extra*1day*')

df_metrics =None
for experiment_dir in experiment_dirs:
    experiment = experiment_dir.split("/")[-1]
    experiment_out_dir = f'{out_result_dir}/{experiment}'
    if not os.path.exists(experiment_out_dir):
        os.makedirs(experiment_out_dir)

    printm(f"## --------------------------- Started Experiment: {experiment} ----------------------------")
    # Compile results into ts dict
    printm("### fetch results for experiment")
    if True:
        result_file = sorted(glob.glob(f"{overall_cache_dir}/{experiment}/results/{result_summary_prefix}*.json"))[-1]
        ts_file = sorted(glob.glob(f"{overall_cache_dir}/{experiment}/results/{ts_prediction_file_prefix}*.pb"))[-1]
        print(result_file)
        exp_results = json.load(open(result_file, 'r'))
        ts_results = pickle.load(open(ts_file, 'rb'))
        cluster_centers = exp_results['direct_labels']
        dataset = exp_results['run_config']['dataset']

    # Filter out only test instances from ts_results
    for id in ts_results.keys():
        print(f"id:{id}:{ts_results[id].shape}")
        ts_results[id] = ts_results[id][ts_results[id].isTrain==False]
        print(f"id:{id}:{ts_results[id].shape}")

    printm("### get cluster labels and best representation accuracy")
    if True:
        direct_cluster_centers = exp_results['direct_labels']
        direct_cluster_labels = get_cluster_labels(direct_cluster_centers, dataset, label_context_v1)
        decoded_cluster_centers = exp_results['decoded_labels']
        decoded_cluster_labels = get_cluster_labels(decoded_cluster_centers, dataset, label_context_v1)
        cluster_labels = []
        for idx in range(len(decoded_cluster_labels)):
            if len(decoded_cluster_labels[idx]) > 0:
                cluster_labels.append(decoded_cluster_labels[idx])
            else:
                cluster_labels.append(direct_cluster_labels[idx])
        df_cluster_merge = pd.DataFrame(np.array([[f"{idx}:"+','.join(cr) for idx,cr in enumerate(direct_cluster_labels)],
                           [f"{idx}:"+','.join(cr) for idx,cr in enumerate(decoded_cluster_labels)],
                           [f"{idx}:"+','.join(cr) for idx,cr in enumerate(cluster_labels)]]).T,columns=['direct','decoded','combination_1'])
        df_cluster_merge.to_csv(f"{experiment_out_dir}/cluster_merge.csv")
        representation_acc = exp_results['repr_training_metrics'][-1]
        json.dump(representation_acc, open(f"{experiment_out_dir}/representation_accuracy.json","w"))

    printm("### compile GT, Onto and Temporal results together")
    compiled_results_ts_dict  = compile_ts_results_v2(ts_results, df_onto_pred,df_gt, cluster_labels)

    # get all available context from onto, gt and temporal
    printm("### get all available contexts")
    if True:
        all_context_list = []
        for id in compiled_results_ts_dict.keys():
            df_ts_id = compiled_results_ts_dict[id]
            all_context_list += np.unique(df_ts_id['gt_context'].values).tolist()
            all_context_list += np.unique(df_ts_id['pred_context'].values).tolist()
            all_context_list += np.unique(df_ts_id['onto_context'].values).tolist()
        all_context_list = np.unique(all_context_list).tolist()
        all_context_list = [xr.split(",") for xr in all_context_list]
        all_context_list = sorted(np.unique(np.concatenate(all_context_list)).tolist())
        if '' in all_context_list:
            del all_context_list[all_context_list.index('')]
        if 'Unknown' in all_context_list:
            del all_context_list[all_context_list.index('Unknown')]
        json.dump(all_context_list, open(f"{experiment_out_dir}/all_contexts.json","w"))

    # Get metrics
    printm("### get timestamp level metrics")
    if True:
        metric_columns = ['gt_vec','onto_pred_vec','tp_pred_vec','combined_pred_vec']
        for id_key in compiled_results_ts_dict.keys():
            dft = compiled_results_ts_dict[id_key]
            dft['gt_vec'] = dft.apply(lambda row: get_ctx_vec(row['gt_context'], all_context_list),axis=1)
            dft['onto_pred_vec'] = dft.apply(lambda row: get_ctx_vec(row['onto_context'], all_context_list),axis=1)
            dft['tp_pred_vec'] = dft.apply(lambda row: get_ctx_vec(row['pred_context'], all_context_list),axis=1)
            dft['combined_pred_vec'] = dft.apply(lambda row: get_ctx_vec(row['combined_context'], all_context_list),axis=1)

        ts_metrics = np.vstack([compiled_results_ts_dict[id][metric_columns].values for id in compiled_results_ts_dict.keys()])
        df_ts_metrics = pd.DataFrame(ts_metrics,columns=metric_columns)
        pickle.dump(compiled_results_ts_dict,open(f"{experiment_out_dir}/compiled_results.pb","wb"))

    # Get overall accuracy metrics for timestamps
    printm("### get overall accuracy metrics")
    if True:
        gt_ts_arr = np.stack(df_ts_metrics['gt_vec'].values)
        onto_ts_arr = np.stack(df_ts_metrics['onto_pred_vec'].values)
        tp_ts_arr = np.stack(df_ts_metrics['tp_pred_vec'].values)
        combined_ts_arr = np.stack(df_ts_metrics['combined_pred_vec'].values)
        onto_ts_metrics = get_overall_metrics(gt_ts_arr, onto_ts_arr)
        tp_ts_metrics = get_overall_metrics(gt_ts_arr, tp_ts_arr)
        combined_ts_metrics = get_overall_metrics(gt_ts_arr,combined_ts_arr)
        df_overall_ts_metrics = pd.DataFrame([onto_ts_metrics,tp_ts_metrics,combined_ts_metrics],columns=['F1','PPV','TPR','JC'],index=['onto','temporal','combined'])
        df_overall_ts_metrics.to_csv(f"{experiment_out_dir}/overall_metrics.csv")
        print(df_overall_ts_metrics)
    # context level accuracy
    printm("### get context level accuracy metrics")
    if True:
        df_context_ts_metrics = pd.DataFrame(all_context_list, columns=['context'])
        df_context_ts_metrics['ppv_onto'] = precision_score(gt_ts_arr, onto_ts_arr, average=None,zero_division=0).round(4)*100
        df_context_ts_metrics['tpr_onto'] = recall_score(gt_ts_arr, onto_ts_arr, average=None,zero_division=0).round(4)*100
        df_context_ts_metrics['ppv_temporal'] = precision_score(gt_ts_arr, tp_ts_arr, average=None,zero_division=0).round(4)*100
        df_context_ts_metrics['tpr_temporal'] = recall_score(gt_ts_arr, tp_ts_arr, average=None,zero_division=0).round(4)*100
        df_context_ts_metrics['ppv_combined'] = precision_score(gt_ts_arr, combined_ts_arr, average=None,zero_division=0).round(4)*100
        df_context_ts_metrics['tpr_combined'] = recall_score(gt_ts_arr, combined_ts_arr, average=None,zero_division=0).round(4)*100
        df_context_ts_metrics.to_csv(f"{experiment_out_dir}/context_metrics.csv")
        print(df_context_ts_metrics)

    printm(f"## Finished Experiment {experiment}")




## --------------------------- Started Experiment: extrasensory_0E6184E1-90C0-48EE-B25A-F1ECB7B9714E_1days_5_1_TAE ----------------------------

### fetch results for experiment

../../cache/sep12_inc//extrasensory_0E6184E1-90C0-48EE-B25A-F1ECB7B9714E_1days_5_1_TAE/results/results_summary_20220914_162525.json
id:0E6184E1-90C0-48EE-B25A-F1ECB7B9714E:(1759, 5)
id:0E6184E1-90C0-48EE-B25A-F1ECB7B9714E:(1494, 5)


### get cluster labels and best representation accuracy

### compile GT, Onto and Temporal results together

...

/var/folders/tj/q4_hs7215pl0xywz58dv85l00000gp/T/ipykernel_1390/1613021110.py:113: RuntimeWarning: invalid value encountered in true_divide
  jc_samples = gt_eq_pred/gt_or_pred


             F1    PPV    TPR     JC
onto       1.16  82.04   0.58   0.67
temporal  41.64  38.34  45.70  40.87
combined  41.64  38.34  45.70  40.87


### get context level accuracy metrics

          context  ppv_onto  tpr_onto  ppv_temporal  tpr_temporal  \
0       Amusement     80.18      0.31         83.12        100.00   
1        ComingIn      0.00      0.00          0.00          0.00   
2       Commuting      0.00      0.00          0.00          0.00   
3        GoingOut      0.00      0.00          0.00          0.00   
4      HavingMeal     57.14      0.31         26.13         17.82   
5       HouseWork     71.43      0.59          0.00          0.00   
6      InAMeeting      0.00      0.00          0.00          0.00   
7      Inactivity      0.00      0.00          0.00          0.00   
8      OfficeWork      0.00      0.00          0.00          0.00   
9   PreparingMeal    100.00      0.14          0.00          0.00   
10       Relaxing     88.89      0.24          0.00          0.00   
11       Sleeping     98.60      1.63          0.00          0.00   
12  UsingBathroom    100.00      0.11          0.00          0.00   

    ppv_combined  tpr_combined  


## Finished Experiment extrasensory_0E6184E1-90C0-48EE-B25A-F1ECB7B9714E_1days_5_1_TAE

In [23]:
exp_results.keys()

dict_keys(['run_config', 'repr_training_metrics', 'data_embeddings', 'cluster_representations', 'instance_predictions', 'context_representations', 'context_positional_counts', 'direct_labels', 'onto_labels', 'decoded_representations', 'decoded_labels'])

In [26]:
ts_results['csh101'].isTrain.value_counts()

False    4660
True      639
Name: isTrain, dtype: int64